In [1]:
import utils

import cv2
import math
import time
import numpy as np
import mediapipe as mp
from scipy.spatial import distance as dist

### model class 

In [25]:
class detections:
    
    def __init__(self):
       
        ### 변수
        self.blink_flag = 0                # 눈 깜빡임 flag    
        self.left_ear = 0                  # 현재 left_ear
        self.right_ear = 0                 # 현재 right_ear
        self.moment_change_flag = 0        # blink_flag 0 -> 1로 변하는 시점
        self.moment_check = round((time.time()),4)    # 매 센싱 시각
        self.PERCLOS = 0
        self.level_change_flag = False

        self.total_list = [[self.moment_check, self.left_ear, self.right_ear, self.blink_flag]]  # 커맨드 리스트
            
        self.level = 0        # 현재 레벨
        self.level_max = 4    # max 레벨
        self.level_min = 0    # min 레벨
                
        self.default_blink = 0.5              # 눈 감은 시간 0.5 이하면 버림
        self.default_EAR_ratio = 0.25         # EAR 기준
        self.default_perclos_time = 60        # perclos 기준 시간
        self.default_perclos_ratio = 0.1      # perclos 비율 기준
        self.default_recovery_time = 140      # recovery 기준 시간
        self.default_longblink_time = 2       # longblink 기준 시간
        self.default_sensing_temp = 1         # 몇 초 단위로 센싱할지


    def index_2d(self, myList, value):
        for i in range(len(myList)):
            if myList[i][0] == value:
                return i
        
        
    def total_list_append(self,):
        tmp_list = [self.moment_check, self.left_ear, self.right_ear, self.blink_flag]
        self.total_list.append(tmp_list)
        return
    
    
    def landmarks_detection(self, img, results, landmark='All', draw=False):

        img_height, img_width = img.shape[:2]
        mesh_coord = [(int(point.x * img_width), int(point.y * img_height)) for point in results.multi_face_landmarks[0].landmark]

        if landmark == 'All':
            if draw:
                [cv2.circle(img, p, 2, utils.GREEN, -1) for p in mesh_coord]
            return mesh_coord

        else:
            mark_coords = [mesh_coord[p] for p in landmark]
            if draw:
                [cv2.circle(img, p, 2, utils.RED, -1) for p in mark_coords]
            return mark_coords
             
                              
    def calculate_ear(self, left_eye_coords, right_eye_coords):

        # 눈 종횡비 계산
        import math
        from scipy.spatial import distance as dist

        left_v = dist.euclidean(left_eye_coords[4], left_eye_coords[12])
        right_v = dist.euclidean(right_eye_coords[4], right_eye_coords[12])
        
        left_h = dist.euclidean(left_eye_coords[0], left_eye_coords[8])
        right_h = dist.euclidean(right_eye_coords[0], right_eye_coords[8])

        left_ear = round((left_v/left_h),4); right_ear = round((right_v/right_h),4)

        # 눈 떴을 때 0, 감으면 1
        if (left_ear < self.default_EAR_ratio and right_ear < self.default_EAR_ratio):
            self.moment_check = round(time.time(),4)
            self.left_ear = left_ear
            self.right_ear = right_ear
            self.blink_flag = 1
            self.total_list_append()
            return
        else:
            self.moment_check = round(time.time(),4)
            self.left_ear = left_ear
            self.right_ear = right_ear
            self.blink_flag = 0
            self.total_list_append()
            return
        

    def manage_drowsy(self):
        
        # 기존 flag vs 현재 flag 비교
        now_flag = self.total_list[-1][-1]
        before_flag = self.total_list[-2][-1]
        
        # case 4개 
        # 0 -> 0 recovery 판단. -> 140초동안 아무일이 없었냐...
        # 0 -> 1 그 순간 체크 (total_list 맨 처음 요소)
        # 1 -> 1 롱블링크 판단.   ( 롱블링크가 안됬다면 -> perclos 판단 )
        # 1 -> 0 블링크 0.5 판단, 롱블링크 판단. perclos 판단.
    
        if before_flag == 0 and now_flag == 0:
            self.judge_recovery()
            return
        
        elif before_flag == 0 and now_flag == 1:
            self.moment_change_flag = self.total_list[-1][0]
            return
            
        elif before_flag == 1 and now_flag == 1:
            close_time = self.total_list[-1][0] - self.moment_change_flag
            if close_time >= self.default_longblink_time:
                print('롱 블링크 발생, 즉시 레벨업')
                self.level_up()
            return
        
        else: # 1 -> 0
            close_time = self.total_list[-1][0] - self.moment_change_flag
            if close_time <= self.default_blink:
                #이때 1인 애들은 다  0으로 바꿔도 무방.. 하다?
                start_idx = self.index_2d(self.total_list, self.moment_change_flag)
                for i in range(start_idx,len(self.total_list),1):
                    self.total_list[i][-1] = 0
                    
            elif close_time >= self.default_longblink_time:
                print('롱 블링크 발생, 즉시 레벨업')
                self.level_up()
            
            self.calculate_perclos()
            return
    

    def sum_blink_time(self, idx):
        
        tmp_list = self.total_list[idx:]
        tmp_list.insert(0, [0,0,0,0])
        
        total_time = 0
        checkss = 0
        
        #print('현재 누적 리스트')
        #for zzzzz in tmp_list:
        #    print(zzzzz)
            
        #print("리스트 총 길이 :",len(tmp_list))
        for idx in range(1, (len(tmp_list)), 1):
            
            before = tmp_list[idx-1][-1]
            now = tmp_list[idx][-1]
            
            if before == 0 and now == 0 :
                continue
            elif before == 1 and now == 1 :
                continue
            elif before == 0 and now == 1 :
                checkss = tmp_list[idx][0]
                #print("체크 시각 :",checkss)
                continue
            else: # 1 -> 0
                tmp_time = round((tmp_list[idx-1][0] - checkss), 4)
                #print("눈 감은 시간 계산 :",tmp_time)

                total_time = round((total_time + tmp_time),4)
                #print("총 눈 감은 시간 계산 :",total_time)

        return total_time
    
    
    def calculate_perclos(self):

        now = self.total_list[-1][0]
        
        for n in range((len(self.total_list)-2),-1,-1):
            if (now - self.total_list[n][0]) <= self.default_perclos_time:
                pass
            else:
                break
        
        standard = self.total_list[n][0]
        
        #print("시작 시각 :",standard)
        #print("지금 시각 :",now)
        
        start_idx = self.index_2d(self.total_list, standard)
        
        total_clos_time = self.sum_blink_time(idx=start_idx)
        
        print("총 눈 감은 시간 :",total_clos_time)

        self.PERCLOS = round((total_clos_time / self.default_perclos_time),2) # perclos 계산
        
        if self.PERCLOS > self.default_perclos_ratio :
            print('Perclos 초과, 레벨업')
            self.level_up()

        return
       
        
    # recovery는 지금부터 140초 이상 됬는데 레벨업이 없었다면 한단계 회복하면 된다.
    def judge_recovery(self):
        
        now = self.total_list[-1][0]
        first = self.total_list[0][0]
        
        if (now - first >= self.default_recovery_time):
            print('정상상태 지속됨, 레벨 다운')
            self.level_down()
        return    

    def level_down(self):# 레벨 0까지 안내려가도록 레벨다운
        if self.level == self.level_min :
            print('[안전] 이미 MIN 레벨')
            self.level = self.level_min
            self.reset()
        else:
            now_level = self.level - 1
            self.reset()
            self.level = now_level
        return


    def level_up(self):
        if self.level == self.level_max :
            print('[경고] MAX 레벨을 넘어섬!!!!!')
            self.reset()
            self.level = self.level_max
        else:
            now_level = self.level + 1
            self.reset()
            self.level = now_level
        return
    
    def reset(self):
        self.__init__()
        return
    
    

### main 문

In [26]:
mp_face_mesh = mp.solutions.face_mesh

frame_counter = 0

cap = cv2.VideoCapture(0)

FONTS = cv2.FONT_HERSHEY_COMPLEX

dsm_model = detections()

# 총 16
left_eye = [362,382,381,380,374,373,390,249,263,466,388,387,386,385,384,398]
right_eye = [33,7,163,144,145,153,154,155,133,173,157,158,159,160,161,246] 

In [27]:
with mp_face_mesh.FaceMesh(min_detection_confidence=0.5, min_tracking_confidence=0.5) as face_mesh:
    
    start_time = time.time()
    
    while True:
        
        frame_counter += 1
        
        ret, frame = cap.read()
        if not ret:
            break
        
        
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)
        results = face_mesh.process(rgb_frame)
        parameter_img = np.zeros((512, 512, 3), np.uint8)
        
        
        if results.multi_face_landmarks:
            # 눈 색 채워서 표시하기
            #frame = utils.fillPolyTrans(frame, [all_mesh_coords[p] for p in left_eye], utils.GREEN, opacity=0.8)
            #frame = utils.fillPolyTrans(frame, [all_mesh_coords[p] for p in right_eye], utils.GREEN, opacity=0.8)
            
            
            ########################################## 튜닝 공간  ##########################################
            ################################################################################################
            ################################################################################################
            
            # 1. 눈 좌표 추출            
            left_eye_coords = dsm_model.landmarks_detection(frame, results, landmark=left_eye, draw=True)
            right_eye_coords = dsm_model.landmarks_detection(frame, results, landmark=right_eye, draw=True)
            
            # 2. ear계산 및 눈 깜빡임 판단
            dsm_model.calculate_ear(left_eye_coords,right_eye_coords)
            
            # 3. 눈 깜빡임 변수로 졸음 판단.
            dsm_model.manage_drowsy()        
            
            
            #for zz in dsm_model.total_list:
            #    print(zz)
            
            # 띄울 파라미터들
            level = "Level : {}".format(dsm_model.level)
            now_left_ear = "left_ear : {}".format(dsm_model.left_ear)
            now_right_ear = "right_ear : {}".format(dsm_model.right_ear)
            perclos = "PERCLOS : {}".format(dsm_model.PERCLOS)
            #blink = "blink time :{}".format(ds_model.total_blink_time[-1])

            

            # 네모박스에 파라미터 띄우기
            # 방법 : cv2.putText(image_original, text, (cx, cy),cv2.FONT_HERSHEY_DUPLEX, 0.5, (255, 255, 255))
            
            cv2.putText(parameter_img, level, (0, 50), cv2.FONT_HERSHEY_DUPLEX, 1.5, (255, 255, 255),1)
            cv2.putText(parameter_img, now_left_ear, (0, 150), cv2.FONT_HERSHEY_DUPLEX, 1.5, (255, 255, 255),1)
            cv2.putText(parameter_img, now_right_ear, (0, 250), cv2.FONT_HERSHEY_DUPLEX, 1.5, (255, 255, 255),1)
            cv2.putText(parameter_img, perclos, (0, 350), cv2.FONT_HERSHEY_DUPLEX, 1.5, (255, 255, 255),1)
            #cv2.putText(parameter_img, blink, (0, 450), cv2.FONT_HERSHEY_DUPLEX, 1.5, (255, 255, 255),1)
                        
            
            
            ################################################################################################
            ################################################################################################
            ################################################################################################
            
        
        end_time = time.time() - start_time
        fps = frame_counter/end_time
        
        frame = utils.textWithBackground(frame, f'FPS: {round(fps,1)}', FONTS, 1.0, (20,50), bgOpacity=0.9)
        
        cv2.imshow('Face mesh', frame)
        cv2.imshow('parameter', parameter_img)
        
        key = cv2.waitKey(1)
        if key==ord('q') or key==ord('Q'):
            break
    
    cv2.destroyAllWindows()
    

총 눈 감은 시간 : 0
총 눈 감은 시간 : 0
총 눈 감은 시간 : 0
총 눈 감은 시간 : 0
총 눈 감은 시간 : 0.4845
총 눈 감은 시간 : 1.2238
총 눈 감은 시간 : 2.0679
총 눈 감은 시간 : 2.8838
총 눈 감은 시간 : 3.6058
총 눈 감은 시간 : 3.6058
총 눈 감은 시간 : 3.6058
총 눈 감은 시간 : 4.2142
총 눈 감은 시간 : 5.0707
총 눈 감은 시간 : 5.0707
총 눈 감은 시간 : 6.6102
Perclos 초과, 레벨업
총 눈 감은 시간 : 0.9431
총 눈 감은 시간 : 0.9431
총 눈 감은 시간 : 0.9431
총 눈 감은 시간 : 1.9021
총 눈 감은 시간 : 2.3891
총 눈 감은 시간 : 2.3891
총 눈 감은 시간 : 2.3891
총 눈 감은 시간 : 2.3891
롱 블링크 발생, 즉시 레벨업
총 눈 감은 시간 : 0
총 눈 감은 시간 : 0
총 눈 감은 시간 : 0


In [23]:
print(time.time())
print(round(time.time(),4))

1700327222.661912
1700327222.6619


In [19]:
lista = [[0,1],
         [1.13,0],
         [2.3,1],
         [2.34,1],
         [4,1],
         [5,0],
         [6,0],
         [7,1],
         [8,1],
         [9,1],
         [10,0],
         [11,0]]

#lista.index(5, start) 

def index_2d(myList,v):
    for i in range(len(myList)):
        if myList[i][0] == v:
            return i
        
myList = [[1,2],[3,4],[5,6]]

#print(index_2d(lista,3))

start_idx = index_2d(lista,2.34)
print(start_idx)

#tmp_list = lista.copy()

#print(tmp_list)
#for i in range(start_idx,len(tmp_list),1):
    #tmp_list[i][1] = 0
    
#print(tmp_list)

3


In [37]:
zzz = test()

print(zzz.a, zzz.b)

zzz.inputs(1,4)

print(zzz.a, zzz.b)

zzz.reset()

print(zzz.a, zzz.b)


0 0
1 4
0 0


In [7]:
for n in range(5-1,-1,-1):
    print(n)

4
3
2
1
0


In [2]:
list = [[2, 9, 3],
        [2, 9, 3],
        [2, 9, 3],
        [2, 9, 3],
        [2, 9, 3],
        [2, 9, 3],
        [2, 9, 3]]
list.insert(0, [0,0,0,0])
print(list)

list.pop(0)
print(list)


[[0, 0, 0, 0], [2, 9, 3], [2, 9, 3], [2, 9, 3], [2, 9, 3], [2, 9, 3], [2, 9, 3], [2, 9, 3]]
[[2, 9, 3], [2, 9, 3], [2, 9, 3], [2, 9, 3], [2, 9, 3], [2, 9, 3], [2, 9, 3]]
